# This is just a notebook to visualise 1kHz filtered raw data

In [1]:
import quantities as pq
import numpy as np
import neo
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
import scipy
from scipy import interpolate
from scipy import fftpack
from scipy import signal
import fnmatch
import os
import mmap
import xarray as xr
import dask.array as da
%matplotlib widget
from IPython.display import display
from ipyfilechooser import FileChooser

from ephyviewer import mkQApp, MainViewer, TraceViewer

In [2]:
def other_mmap(filename, shape, dtype, chunks):
    with open(filename, "rb") as fh:
        # `mmap` duplicates the file descriptor
        # `0` means map the full file
        mm = mmap.mmap(fh.fileno(), 0, access=mmap.ACCESS_READ)

    # coerce to NumPy array of expected type and shape
    a = np.asarray(mm).view(dtype).reshape(shape)

    # Don't call `asarray`
    # name=False to avoid hashing
    a_d = da.from_array(a, chunks=chunks, asarray=False, name=False)
    return a_d

# Function to find files containing a specific string
def find_files_with_string(folder_path, search_string):
    matching_file = []
    # Traverse the folder to find files
    for root, _, files in os.walk(folder_path):
        for file in files:
            if fnmatch.fnmatch(file, f"*{search_string}*"):
                matching_file=os.path.join(root, file)
    return matching_file

In [4]:
dpath = "//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"
try:
    %store -r dpath
except:
    print("data not in strore")
    dpath ="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording"

# Set up Initial Basic Parameters#
minian_path = "."

fc1 = FileChooser(dpath,select_default=True, show_only_dirs = True, title = "<b>Folder with videos</b>")
display(fc1)

# Sample callback function
def update_my_folder(chooser):
    global dpath
    dpath = chooser.selected
    %store dpath
    return 

# Register callback function
fc1.register_callback(update_my_folder)

FileChooser(path='\\10.69.168.1\crnldata\waking\audrey_hay\CamilleData\Tests\testsMay16th2024\Expe_2024-05-16_…

Stored 'dpath' (str)


Load LFPs data

In [5]:
folderpath = Path(dpath)
numchannels=32

if find_files_with_string(folderpath,  ".bin"): #Bonsai
    matching_file = find_files_with_string(folderpath, ".bin")
    All = np.fromfile(matching_file, dtype=np.uint16)
    All = All - int(65535/2)
    All = All.astype(np.int16)
    if 'sommeil' in dpath:
        numchannels=64
    All = All.reshape(-1,numchannels)
    print('File loaded: OE32channels.bin,', numchannels, 'channels')
elif find_files_with_string(folderpath,  "continuous.dat"): #OpenEphys
    matching_file = find_files_with_string(folderpath, "continuous.dat")
    All = np.fromfile(matching_file, dtype="int16")
    if 'sommeil' in dpath:
        numchannels=64
    All = All.reshape(-1,numchannels)
    print('File loaded: continuous.dat,', numchannels, 'channels')
elif find_files_with_string(folderpath,  "RawDataChannelExtractedDS.npy"): #OpenEphys Gaelle's Data
    matching_file = find_files_with_string(folderpath, "RawDataChannelExtractedDS.npy")
    All = np.load(matching_file, mmap_mode= 'r')
    All = All.reshape(-1,numchannels)
    print('File loaded: RawDataChannelExtractedDS.npy,', numchannels, 'channels')

File loaded: OE32channels.bin, 32 channels


In [6]:
#BOB
S1=All[:, 5]-All[:,6]
PFC=All[:, 2]-All[:,3]
CA1=All[:, 29]-All[:,30]
EMG=All[:,27]
combined = np.stack([S1, PFC, CA1, EMG/10], axis = 1)

Visualise All

In [7]:
from ephyviewer import mkQApp, MainViewer, TraceViewer, TimeFreqViewer
from ephyviewer import InMemoryAnalogSignalSource
import ephyviewer
app = mkQApp()

sample_rate = 1000.
t_start = 0.

#Create the main window that can contain several viewers
win = MainViewer()
#view1 = TraceViewer.from_numpy(All, sample_rate, t_start, 'Signals')

source = InMemoryAnalogSignalSource(combined, sample_rate, t_start)
view1 = TraceViewer(source=source, name='trace')

view1.params['display_labels'] = True
view1.params['scale_mode'] = 'same_for_all'
view1.auto_scale()

#create a time freq viewer conencted to the same source
view2 = TimeFreqViewer(source=source, name='tfr')

view2.params['show_axis'] = True
view2.params['timefreq', 'deltafreq'] = 1
view2.params['timefreq', 'f_stop'] = 400
view2.by_channel_params['ch0', 'visible'] = False
view2.by_channel_params['ch1', 'visible'] = True
view2.by_channel_params['ch1', 'clim'] = 1

#add them to mainwindow
win.add_view(view1)
win.add_view(view2)

#Run
win.show()
app.exec_()


viewer has moved already 1 7.534079999999999 6.780672
viewer has moved already 1 11.30112 9.794304
viewer has moved already 1 11.30112 10.547712
viewer has moved already 1 14.314752 13.561344
viewer has moved already 1 15.821568 15.068159999999999
viewer has moved already 1 18.8352 17.328384
viewer has moved already 1 18.8352 18.081792
viewer has moved already 1 21.848831999999998 21.095424
viewer has moved already 1 22.60224 21.848831999999998
viewer has moved already 1 25.615872 24.862464
viewer has moved already 1 30.889727999999998 30.136319999999998
viewer has moved already 1 35.41017599999999 34.656768
viewer has moved already 1 39.177216 38.423808
viewer has moved already 1 40.684032 39.930624
viewer has moved already 1 43.697663999999996 42.190848
viewer has moved already 1 43.697663999999996 42.944256
viewer has moved already 1 47.464704 45.957888
viewer has moved already 1 47.464704 46.711296
viewer has moved already 1 48.97152 48.218112
viewer has moved already 1 51.985152 5

0

In [43]:
x=All[:20000,0]
X = fft(x)
N = len(X)
n = np.arange(N)
T = N/sr
freq = n/T 

plt.close()
plt.figure(figsize = (12, 6))
plt.subplot(121)

plt.stem(freq, np.abs(X), 'b', \
         markerfmt=" ", basefmt="-b")
plt.xlabel('Freq (Hz)')
plt.ylabel('FFT Amplitude |X(freq)|')


NameError: name 'fft' is not defined